# **getINNOtized-BI-Solution**

## **Business Understanding**

## **Data Understanding**
N|B the pip verion used in this project -pip-24.2

### **Expolatory Data Analysis**

## **Data Preparation**

### **Answering Analytical Questions** 

### **Findings and Insights**

### **Reccommendations** 

## **Deployment**